# Comparison between my code and bibliography: The Journal of Physical Chemistry Letters
## Vol 11/Issue 15
### Local Enhancement of Dynamic Correlation in Excited States: Fresh Perspective on Ionicity and Development of Correlation Density Functional Approximation Based on the On-Top Pair Density

**Article Subscribed: June 26, 2020**  
*Authors: Michał Hapka, Katarzyna Pernal, Oleg V. Gritsenko*

In [1]:
# Imports
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
sys.path.append("src")

from file_management import FileManager
from cluster_connection import ClusterConnection
from job_manager import JobManager
from slurm_manager import SlurmManager
from flux_correlation_indicator import FluxCorrelationIndicator
from molecule import Molecule
from basis import BasisSet
from method import Method
from input_specification import InputSpecification

In [2]:
def interpolate_molecular_indicators(df, molecular_geometry, atoms_of_interest=['C']):
   # Calculate grid spacing from data
   x_sorted = sorted(df['x'].unique())
   y_sorted = sorted(df['y'].unique())
   z_sorted = sorted(df['z'].unique())
   
   dx = x_sorted[1] - x_sorted[0]
   dy = y_sorted[1] - y_sorted[0]
   dz = z_sorted[1] - z_sorted[0]
   
   grid_spacing = min(dx, dy, dz)
   tolerance = grid_spacing/2

   def find_nearest_points(y_target, z_target, x_slice):
       y_vals = x_slice['y'].unique()
       z_vals = x_slice['z'].unique()
       
       y0 = max(y_vals[y_vals <= y_target]) if any(y_vals <= y_target) else min(y_vals)
       y1 = min(y_vals[y_vals >= y_target]) if any(y_vals >= y_target) else max(y_vals)
       z0 = max(z_vals[z_vals <= z_target]) if any(z_vals >= z_target) else min(z_vals)
       z1 = min(z_vals[z_vals >= z_target]) if any(z_vals >= z_target) else max(z_vals)
       
       return y0, y1, z0, z1

   def bilinear_interpolate(y, z, points, y0, y1, z0, z1, quantity):
       if y0 == y1: return np.mean(points[quantity].values[[0,2]]) if z0 != z1 else points[quantity].values[0]
       if z0 == z1: return np.mean(points[quantity].values[[0,1]]) if y0 != y1 else points[quantity].values[0]
       
       wy = (y - y0)/(y1 - y0)
       wz = (z - z0)/(z1 - z0)
       vals = points[quantity].values
       
       return (1-wy)*(1-wz)*vals[0] + wy*(1-wz)*vals[1] + \
              (1-wy)*wz*vals[2] + wy*wz*vals[3]

   atoms = [atom for atom in molecular_geometry['optimized_geometry'] 
            if atom['symbol'] in atoms_of_interest][:len(atoms_of_interest)//2 + 1]
           
   results = {}
   for atom in atoms:
       y_atom, z_atom = atom['coordinates'][1:]
       interpolated_values = []
       
       for x in x_sorted:
           x_slice = df[abs(df['x'] - x) < tolerance]
           if x_slice.empty:
               continue
               
           y0, y1, z0, z1 = find_nearest_points(y_atom, z_atom, x_slice)
           points = x_slice[
               ((abs(x_slice['y'] - y0) < tolerance) | (abs(x_slice['y'] - y1) < tolerance)) &
               ((abs(x_slice['z'] - z0) < tolerance) | (abs(x_slice['z'] - z1) < tolerance))
           ]
           
           if len(points) >= 4:
               density = bilinear_interpolate(y_atom, z_atom, points, y0, y1, z0, z1, 'density')
               ontop = bilinear_interpolate(y_atom, z_atom, points, y0, y1, z0, z1, 'on_top')
               interpolated_values.append({
                   'x': x - atom['coordinates'][0],
                   'indicator': (2 * ontop) / density**2
               })
       
       if interpolated_values:
           results[f"{atom['symbol']}_{len(results)+1}"] = pd.DataFrame(interpolated_values)

   atom_dfs = []
   for atom_id, atom_df in results.items():
       atom_df = atom_df.rename(columns={'indicator': f'X{atom_id}'})
       atom_dfs.append(atom_df.set_index('x'))
   
   # Combine all DataFrames on x-coordinate
   combined_df = pd.concat(atom_dfs, axis=1).reset_index()
   return combined_df

In [3]:
def launch_ontop_calculation(molecule_name, method_name, basis_name):
        # Code to define molecule, method, and basis, then execute job and retrieve results
    
    # Placeholder: Move to project root if needed
    # os.chdir("..")  # Uncomment if needed to change directory
    
    # Define molecule, method, basis
    molecule = Molecule(name=molecule_name)  # Example placeholder
    method = Method(method_name)
    basis = BasisSet(basis_name)
    import re


# Regular expression to match parts between ( and , and between , and )
    match = re.search(r'\(([^,]+),\s*([^)]+)\)', method_name)
    if match:
        part1 = match.group(1).strip()
        part2 = match.group(2).strip()

    
    job_name = molecule_name[:2]+part1+part2+basis_name.replace('-','')

    
    # Initialize dataframe to None
    df = None
    with ClusterConnection(config_file="utils/cluster_config.json") as connection:
        file_manager = FileManager(connection)
        job_manager = JobManager(connection, file_manager, SlurmManager())
        flux_manager = FluxCorrelationIndicator(connection, file_manager, job_manager)
    
        # Define the job name and execute the flux
        calculation_results = flux_manager.handle_flux(job_name, molecule, method, basis)
        df = file_manager.get_results(job_name)
        df = interpolate_molecular_indicators(df,calculation_results)
        return df, calculation_results


In [4]:
# Function to obtain atomic indicator
def obtain_atomic_indicator(df):
    # Filter data where y and z are approximately 0

    # Convert x and y from Hartree to Ångstroms
    df['x'] = df['x'] * 0.529177
    df['y'] = df['y'] * 0.529177
    
    # Create indicator column
    df['indicator'] = 2 * df['on_top'] / (df['density']) ** 2
    
    # Select relevant columns
    result_df = df[['x', 'y', 'indicator']].dropna(subset=['indicator'])
    return result_df 


In [5]:
def plot_comparison(molecule_name, calculated_df, limit=None, reference_path='./notebooks/figures/'):
   """
   Displays reference image and calculated X(r) plot side by side.
   
   Parameters:
   - molecule_name (str): Name of molecule (e.g., 'ethene', 'butadiene')
   - calculated_df (pd.DataFrame): DataFrame with 'x' and 'XC_n' columns
   - limit (float): x-axis limit for plotting
   """
   
   # Load the reference image
   img_path = f'./notebooks/figures/{molecule.lower()}.png'
   try:
       reference_img = Image.open(img_path)
   except FileNotFoundError:
       print(f"Reference image for {molecule} not found at {img_path}. Placeholder will be shown.")
       reference_img = None

   # Filter data
   if limit:
       calculated_df = calculated_df[calculated_df['x'] <= limit]

   # Create figure
   fig = plt.figure(figsize=(14, 6))
   fig.suptitle(f'X(r) Comparison for {molecule_name}', fontsize=16)

   # Left: Reference
   ax1 = fig.add_subplot(121)
   if reference_img is not None:
       ax1.imshow(reference_img)
       ax1.axis('off')
       ax1.set_title('Bibliography Results')
   else:
       ax1.text(0.5, 0.5, 'Reference Image Not Available', 
               ha='center', va='center', fontsize=12)
       ax1.axis('off')

   # Right: Calculated X(r)
   ax2 = fig.add_subplot(122)
   x_cols = [col for col in calculated_df.columns if col.startswith('X')]
   for col in x_cols:
       ax2.plot(calculated_df['x'], calculated_df[col], 
               label=f'Carbon {col[2:]}', linewidth=1.5)

   ax2.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5)
   ax2.set_xlabel('r (Å)')
   ax2.set_ylabel('X(r)')
   ax2.set_title('Calculated Results')
   ax2.grid(True, alpha=0.3)
   ax2.legend()

   plt.tight_layout(rect=[0, 0, 1, 0.96])
   return fig

## Atoms

### C2H4

In [6]:
molecule = 'ethene'
df = launch_ontop_calculation(molecule,'CASSCF(2,2)','TZVP');

Geometry for ethene successfully loaded.
Connected to atlas.
Gaussian input file './test/et22TZVP.com' generated successfully.
Command output: 
Uploaded test/et22TZVP.com to /dipc/javidom/proyect-3-indicator/et22TZVP/et22TZVP.com on the cluster.
Uploaded slurm_scripts/et22TZVP.slurm to /dipc/javidom/proyect-3-indicator/et22TZVP/et22TZVP.slurm on the cluster.
Command output: 
Command output: 
Command output: 
Command output: 
Command output: 
Command output: Submitted batch job 2210524

Command output:                JOBID   PARTITION         QOS                  NAME          USER    ST         TIME  NODES  NODELIST(REASON)
             2210524     general     regular              et22TZVP       javidom    PD         0:00      1  (Priority)

Command output:                JOBID   PARTITION         QOS                  NAME          USER    ST         TIME  NODES  NODELIST(REASON)
             2210524     general     regular              et22TZVP       javidom    PD         0:00      1 

KeyError: 'z'

In [ ]:
plot_comparison(molecule, df,limit=1)

### C4H6

In [ ]:
molecule = 'butadiene'
df = launch_ontop_calculation(molecule,'CASSCF(4,4)','TZVP');

In [ ]:
plot_comparison(molecule, obtain_atomic_indicator(df),limit=1.09)

### C6H8

In [ ]:
molecule = 'hexatriene'
df = launch_ontop_calculation(molecule,'CASSCF(6,6)','TZVP');

In [ ]:
plot_comparison(molecule, obtain_atomic_indicator(df),limit=1.59)

In [ ]:
## Molecules